# 🧬 Módulo 2: Representación y Visualización Molecular
## Actividad 2.5: Generación de Conformaciones Moleculares

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_02_representacion_molecular/05_conformaciones.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender los conceptos fundamentales de conformaciones moleculares
- Generar múltiples conformaciones usando RDKit
- Aplicar métodos de búsqueda conformacional (sistemático, estocástico)
- Optimizar geometrías con campos de fuerza
- Analizar superficies de energía potencial
- Calcular distribuciones de Boltzmann
- Identificar conformaciones de baja energía

---

## 1. Instalación de Dependencias

In [ ]:
# Instalación en Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit py3Dmol -q

print("✓ Dependencias instaladas")

In [ ]:
# Importar librerías necesarias
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors
from rdkit.Chem import Draw
import py3Dmol
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

print("✓ Librerías importadas correctamente")

## 2. Conceptos Fundamentales

### ¿Qué son las Conformaciones?

Las **conformaciones** son los diferentes arreglos espaciales de una molécula que resultan de la rotación alrededor de enlaces simples. A diferencia de los isómeros, las conformaciones pueden interconvertirse sin romper enlaces.

### Importancia en Química Computacional

- **Propiedades moleculares**: Actividad biológica, energía de enlace, reactividad
- **Docking molecular**: Predicción de unión ligando-proteína
- **QSAR**: Relaciones estructura-actividad cuantitativas
- **Cristalografía**: Comparación con estructuras experimentales

### Superficie de Energía Potencial (PES)

La PES describe la energía de una molécula en función de sus coordenadas:

$$E = E(\theta_1, \theta_2, ..., \theta_n)$$

Donde $\theta_i$ son los ángulos diedros rotables.

## 3. Generación Básica de Conformaciones con RDKit

RDKit utiliza el algoritmo **ETKDG** (Experimental Torsion-angle preference with Distance Geometry) para generar conformaciones.

In [ ]:
# Ejemplo: Butano - molécula simple con rotación interna
butano_smiles = "CCCC"
butano = Chem.MolFromSmiles(butano_smiles)
butano = Chem.AddHs(butano)  # Añadir hidrógenos explícitos

# Generar una conformación 3D
AllChem.EmbedMolecule(butano, randomSeed=42)

# Optimizar con campo de fuerza MMFF94
AllChem.MMFFOptimizeMolecule(butano)

print(f"Molécula: {Chem.MolToSmiles(Chem.RemoveHs(butano))}")
print(f"Número de átomos: {butano.GetNumAtoms()}")
print(f"Número de enlaces rotables: {rdMolDescriptors.CalcNumRotatableBonds(butano)}")

In [ ]:
# Visualización 3D
def visualizar_molecula(mol, estilo='stick', width=400, height=300):
    """Visualiza una molécula en 3D usando py3Dmol"""
    mol_block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=width, height=height)
    viewer.addModel(mol_block, 'mol')
    viewer.setStyle({estilo: {}})
    viewer.zoomTo()
    return viewer.show()

visualizar_molecula(butano)

## 4. Generación de Múltiples Conformaciones

Para explorar el espacio conformacional, generamos múltiples conformaciones y analizamos sus energías.

In [ ]:
def generar_conformaciones(smiles, num_conf=50, optimizar=True):
    """
    Genera múltiples conformaciones de una molécula
    
    Parámetros:
    -----------
    smiles : str
        SMILES de la molécula
    num_conf : int
        Número de conformaciones a generar
    optimizar : bool
        Si True, optimiza cada conformación con MMFF94
    
    Retorna:
    --------
    mol : rdkit.Chem.Mol
        Molécula con múltiples conformaciones
    energias : list
        Lista de energías en kcal/mol
    """
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    # Generar conformaciones con ETKDG
    params = AllChem.ETKDGv3()
    params.randomSeed = 42
    conf_ids = AllChem.EmbedMultipleConfs(mol, numConfs=num_conf, params=params)
    
    energias = []
    
    if optimizar:
        # Optimizar y calcular energías
        props = AllChem.MMFFGetMoleculeProperties(mol)
        for conf_id in conf_ids:
            ff = AllChem.MMFFGetMoleculeForceField(mol, props, confId=conf_id)
            ff.Minimize()
            energia = ff.CalcEnergy()
            energias.append(energia)
    else:
        # Solo calcular energías sin optimizar
        props = AllChem.MMFFGetMoleculeProperties(mol)
        for conf_id in conf_ids:
            ff = AllChem.MMFFGetMoleculeForceField(mol, props, confId=conf_id)
            energias.append(ff.CalcEnergy())
    
    return mol, energias

# Ejemplo con ciclohexano
ciclohexano_smiles = "C1CCCCC1"
mol_conf, energias = generar_conformaciones(ciclohexano_smiles, num_conf=100)

print(f"Conformaciones generadas: {mol_conf.GetNumConformers()}")
print(f"\nEstadísticas de energía (kcal/mol):")
print(f"  Mínima: {min(energias):.2f}")
print(f"  Máxima: {max(energias):.2f}")
print(f"  Promedio: {np.mean(energias):.2f}")
print(f"  Desviación estándar: {np.std(energias):.2f}")

In [ ]:
# Histograma de energías
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(energias, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Energía (kcal/mol)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Energías Conformacionales')
plt.axvline(min(energias), color='red', linestyle='--', label='Mínimo global')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(energias, 'o-', markersize=3, alpha=0.6)
plt.xlabel('Índice de conformación')
plt.ylabel('Energía (kcal/mol)')
plt.title('Energías por Conformación')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Identificación de Conformaciones de Baja Energía

Es crucial identificar y analizar las conformaciones más estables.

In [ ]:
def obtener_conformaciones_top(mol, energias, n_top=5):
    """
    Obtiene las n conformaciones de menor energía
    
    Retorna:
    --------
    indices : list
        Índices de las conformaciones de menor energía
    energias_top : list
        Energías correspondientes
    """
    # Ordenar por energía
    indices_ordenados = np.argsort(energias)
    indices = indices_ordenados[:n_top]
    energias_top = [energias[i] for i in indices]
    
    return indices, energias_top

# Obtener las 5 conformaciones de menor energía
indices_top, energias_top = obtener_conformaciones_top(mol_conf, energias, n_top=5)

print("Top 5 conformaciones de menor energía:\n")
energia_min = min(energias)
for i, (idx, E) in enumerate(zip(indices_top, energias_top), 1):
    deltaE = E - energia_min
    print(f"{i}. Conformación {idx}: E = {E:.2f} kcal/mol (ΔE = {deltaE:.2f} kcal/mol)")

In [ ]:
# Visualizar las 3 conformaciones de menor energía
def visualizar_multiples_conformaciones(mol, conf_ids, energias, title="Conformaciones"):
    """
    Visualiza múltiples conformaciones en una cuadrícula
    """
    viewer = py3Dmol.view(width=800, height=300, viewergrid=(1, len(conf_ids)))
    
    for i, conf_id in enumerate(conf_ids):
        mol_block = Chem.MolToMolBlock(mol, confId=int(conf_id))
        viewer.addModel(mol_block, 'mol', viewer=(0, i))
        viewer.setStyle({'stick': {}}, viewer=(0, i))
        viewer.addLabel(f"E = {energias[conf_id]:.2f} kcal/mol", 
                       {'fontSize': 10, 'fontColor': 'black', 'backgroundColor': 'white'},
                       viewer=(0, i))
        viewer.zoomTo(viewer=(0, i))
    
    return viewer.show()

print("Visualización de las 3 conformaciones de menor energía:\n")
visualizar_multiples_conformaciones(mol_conf, indices_top[:3], energias)

## 6. Distribución de Boltzmann

Las poblaciones conformacionales en equilibrio térmico siguen la distribución de Boltzmann:

$$P_i = \frac{e^{-E_i/RT}}{\sum_j e^{-E_j/RT}}$$

Donde:
- $P_i$ es la población de la conformación $i$
- $E_i$ es la energía de la conformación $i$
- $R$ es la constante de gases (1.987 × 10⁻³ kcal/(mol·K))
- $T$ es la temperatura (típicamente 298.15 K)

In [ ]:
def calcular_poblacion_boltzmann(energias, temperatura=298.15):
    """
    Calcula las poblaciones de Boltzmann para cada conformación
    
    Parámetros:
    -----------
    energias : list
        Energías en kcal/mol
    temperatura : float
        Temperatura en Kelvin
    
    Retorna:
    --------
    poblaciones : np.array
        Poblaciones de Boltzmann (suman 1.0)
    """
    R = 0.001987  # kcal/(mol·K)
    RT = R * temperatura
    
    # Convertir energías relativas
    E_rel = np.array(energias) - min(energias)
    
    # Calcular factores de Boltzmann
    boltzmann = np.exp(-E_rel / RT)
    
    # Normalizar
    poblaciones = boltzmann / np.sum(boltzmann)
    
    return poblaciones

# Calcular poblaciones
poblaciones = calcular_poblacion_boltzmann(energias)

# Mostrar las conformaciones más pobladas
indices_poblacion = np.argsort(poblaciones)[::-1]  # Ordenar descendente

print("Conformaciones más pobladas a 298.15 K:\n")
print(f"{'Rank':<6} {'Conf.':<8} {'Energía':<12} {'ΔE':<12} {'Población':<12}")
print("-" * 60)

E_min = min(energias)
for i, idx in enumerate(indices_poblacion[:10], 1):
    E = energias[idx]
    deltaE = E - E_min
    pop = poblaciones[idx] * 100
    print(f"{i:<6} {idx:<8} {E:<12.2f} {deltaE:<12.2f} {pop:<12.2f}%")

# Población acumulada del top 5
pop_top5 = sum([poblaciones[idx] for idx in indices_poblacion[:5]]) * 100
print(f"\nPoblación acumulada (Top 5): {pop_top5:.1f}%")

In [ ]:
# Visualización de la distribución de Boltzmann
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(energias, poblaciones * 100, alpha=0.6, s=30)
plt.xlabel('Energía (kcal/mol)')
plt.ylabel('Población (%)')
plt.title('Distribución de Boltzmann a 298.15 K')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Gráfico de barras para las top 10 conformaciones
top_10_idx = indices_poblacion[:10]
top_10_pop = [poblaciones[i] * 100 for i in top_10_idx]
plt.bar(range(1, 11), top_10_pop, color='coral', edgecolor='black', alpha=0.7)
plt.xlabel('Ranking de Conformación')
plt.ylabel('Población (%)')
plt.title('Top 10 Conformaciones Más Pobladas')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Análisis de Ángulos Diedros

Los ángulos diedros son clave para entender las conformaciones.

In [ ]:
def obtener_angulos_diedros(mol, conf_id=0):
    """
    Calcula todos los ángulos diedros principales de una conformación
    
    Retorna:
    --------
    diedros : list of tuples
        Lista de (átomos, ángulo en grados)
    """
    diedros = []
    conf = mol.GetConformer(conf_id)
    
    # Encontrar todos los enlaces rotables
    for bond in mol.GetBonds():
        if bond.GetBondType() == Chem.BondType.SINGLE and not bond.IsInRing():
            atom1 = bond.GetBeginAtomIdx()
            atom2 = bond.GetEndAtomIdx()
            
            # Encontrar vecinos para formar el diedro
            vecinos1 = [x.GetIdx() for x in mol.GetAtomWithIdx(atom1).GetNeighbors() if x.GetIdx() != atom2]
            vecinos2 = [x.GetIdx() for x in mol.GetAtomWithIdx(atom2).GetNeighbors() if x.GetIdx() != atom1]
            
            if vecinos1 and vecinos2:
                # Calcular ángulo diedro
                angulo = rdMolTransforms.GetDihedralDeg(conf, vecinos1[0], atom1, atom2, vecinos2[0])
                diedros.append(((vecinos1[0], atom1, atom2, vecinos2[0]), angulo))
    
    return diedros

# Para el análisis de diedros, necesitamos importar rdMolTransforms
from rdkit.Chem import rdMolTransforms

# Ejemplo con butano
butano = Chem.MolFromSmiles("CCCC")
butano = Chem.AddHs(butano)
mol_but, energias_but = generar_conformaciones("CCCC", num_conf=50)

# Analizar el ángulo diedro C-C-C-C en varias conformaciones
print("Análisis de ángulos diedros en butano:\n")
print(f"{'Conf.':<8} {'Diedro C-C-C-C':<18} {'Energía (kcal/mol)':<20}")
print("-" * 50)

for i in indices_top[:5]:
    diedros = obtener_angulos_diedros(mol_but, conf_id=int(i))
    if diedros:
        angulo = diedros[0][1]
        print(f"{i:<8} {angulo:<18.1f}° {energias_but[i]:<20.2f}")

## 8. Exploración Sistemática vs. Estocástica

### Búsqueda Sistemática
- Explora todas las combinaciones de ángulos diedros
- Garantiza encontrar el mínimo global
- Computacionalmente costosa: $O(n^m)$ donde $n$ = incrementos, $m$ = enlaces rotables

### Búsqueda Estocástica (ETKDG)
- Muestreo aleatorio basado en conocimiento químico
- Más eficiente para moléculas grandes
- No garantiza encontrar el mínimo global

In [ ]:
# Comparación de eficiencia
import time

def benchmark_generacion(smiles, metodo='ETKDG', num_conf=50):
    """
    Mide el tiempo de generación de conformaciones
    """
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    inicio = time.time()
    
    if metodo == 'ETKDG':
        params = AllChem.ETKDGv3()
        params.randomSeed = 42
        AllChem.EmbedMultipleConfs(mol, numConfs=num_conf, params=params)
    elif metodo == 'random':
        params = AllChem.ETKDG()
        params.randomSeed = 42
        params.useRandomCoords = True
        AllChem.EmbedMultipleConfs(mol, numConfs=num_conf, params=params)
    
    tiempo = time.time() - inicio
    
    return mol, tiempo

# Probar con ibuprofen
ibuprofen_smiles = "CC(C)Cc1ccc(cc1)C(C)C(=O)O"

print("Benchmark de generación conformacional (Ibuprofeno):\n")

mol_etkdg, tiempo_etkdg = benchmark_generacion(ibuprofen_smiles, 'ETKDG', 100)
print(f"ETKDG v3:")
print(f"  Tiempo: {tiempo_etkdg:.3f} segundos")
print(f"  Conformaciones: {mol_etkdg.GetNumConformers()}")
print(f"  Enlaces rotables: {rdMolDescriptors.CalcNumRotatableBonds(mol_etkdg)}")

## 9. Ejemplo Completo: Análisis Conformacional de un Fármaco

In [ ]:
def analisis_conformacional_completo(smiles, nombre, num_conf=100):
    """
    Realiza un análisis conformacional completo
    """
    print(f"\n{'='*60}")
    print(f"Análisis Conformacional: {nombre}")
    print(f"{'='*60}\n")
    
    # 1. Generar conformaciones
    print("1. Generando conformaciones...")
    mol, energias = generar_conformaciones(smiles, num_conf=num_conf)
    print(f"   ✓ {len(energias)} conformaciones generadas")
    
    # 2. Estadísticas energéticas
    print("\n2. Estadísticas energéticas:")
    print(f"   Energía mínima: {min(energias):.2f} kcal/mol")
    print(f"   Energía máxima: {max(energias):.2f} kcal/mol")
    print(f"   Rango: {max(energias) - min(energias):.2f} kcal/mol")
    print(f"   Promedio: {np.mean(energias):.2f} kcal/mol")
    
    # 3. Identificar conformaciones de baja energía
    print("\n3. Top 5 conformaciones de menor energía:")
    indices, energias_top = obtener_conformaciones_top(mol, energias, n_top=5)
    E_min = min(energias)
    for i, (idx, E) in enumerate(zip(indices, energias_top), 1):
        print(f"   {i}. Conf {idx}: E = {E:.2f} kcal/mol (ΔE = {E - E_min:.2f})")
    
    # 4. Poblaciones de Boltzmann
    print("\n4. Análisis de poblaciones (298.15 K):")
    poblaciones = calcular_poblacion_boltzmann(energias)
    idx_poblacion = np.argsort(poblaciones)[::-1]
    
    pop_acum = 0
    for i, idx in enumerate(idx_poblacion[:5], 1):
        pop = poblaciones[idx] * 100
        pop_acum += pop
        print(f"   {i}. Conf {idx}: {pop:.1f}%")
    print(f"   Población acumulada (Top 5): {pop_acum:.1f}%")
    
    # 5. Conformaciones dentro de 2 kcal/mol del mínimo
    umbral = 2.0
    conf_bajas = [i for i, E in enumerate(energias) if (E - E_min) <= umbral]
    print(f"\n5. Conformaciones dentro de {umbral} kcal/mol del mínimo: {len(conf_bajas)}")
    
    return mol, energias, poblaciones

# Analizar aspirina
aspirina_smiles = "CC(=O)Oc1ccccc1C(=O)O"
mol_asp, energias_asp, poblaciones_asp = analisis_conformacional_completo(
    aspirina_smiles, 
    "Aspirina (Ácido Acetilsalicílico)",
    num_conf=100
)

In [ ]:
# Visualización completa
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Histograma de energías
axes[0, 0].hist(energias_asp, bins=30, color='lightblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(min(energias_asp), color='red', linestyle='--', linewidth=2, label='Mínimo')
axes[0, 0].set_xlabel('Energía (kcal/mol)', fontsize=11)
axes[0, 0].set_ylabel('Frecuencia', fontsize=11)
axes[0, 0].set_title('Distribución de Energías', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Scatter plot energía vs población
axes[0, 1].scatter(energias_asp, poblaciones_asp * 100, alpha=0.6, s=40, c=energias_asp, cmap='viridis')
axes[0, 1].set_xlabel('Energía (kcal/mol)', fontsize=11)
axes[0, 1].set_ylabel('Población (%)', fontsize=11)
axes[0, 1].set_title('Energía vs Población de Boltzmann', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Top 10 poblaciones
idx_top = np.argsort(poblaciones_asp)[::-1][:10]
pop_top = [poblaciones_asp[i] * 100 for i in idx_top]
axes[1, 0].bar(range(1, 11), pop_top, color='coral', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Ranking', fontsize=11)
axes[1, 0].set_ylabel('Población (%)', fontsize=11)
axes[1, 0].set_title('Top 10 Conformaciones', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Energías relativas (ΔE)
E_min = min(energias_asp)
delta_E = [E - E_min for E in energias_asp]
axes[1, 1].hist(delta_E, bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(2.0, color='red', linestyle='--', linewidth=2, label='Umbral 2 kcal/mol')
axes[1, 1].set_xlabel('ΔE (kcal/mol)', fontsize=11)
axes[1, 1].set_ylabel('Frecuencia', fontsize=11)
axes[1, 1].set_title('Energías Relativas al Mínimo', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Análisis Conformacional Completo - Aspirina', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 10. Ejercicios Prácticos

### Ejercicio 1: Análisis de Ciclohexano

El ciclohexano puede adoptar conformaciones de **silla** (más estable) y **bote**. 

**Tarea:**
1. Genera 200 conformaciones de ciclohexano (SMILES: `C1CCCCC1`)
2. Identifica las conformaciones de silla y bote por su energía
3. Calcula la diferencia de energía entre ellas
4. Visualiza ambas conformaciones
5. ¿Cuál es la población relativa de cada conformación a 298 K?

In [ ]:
# TU CÓDIGO AQUÍ
# Genera conformaciones de ciclohexano y analiza silla vs bote

### Ejercicio 2: Rotación de Etano

El etano (C₂H₆) presenta barreras rotacionales entre conformaciones eclipsadas y alternadas.

**Tarea:**
1. Genera 100 conformaciones de etano (SMILES: `CC`)
2. Calcula los ángulos diedros H-C-C-H
3. Grafica energía vs ángulo diedro
4. Identifica la barrera rotacional (diferencia entre conformación eclipsada y alternada)
5. Compara con el valor experimental (~3 kcal/mol)

In [ ]:
# TU CÓDIGO AQUÍ
# Analiza la barrera rotacional del etano

### Ejercicio 3: Análisis de un Fármaco Flexible

Analiza el ibuprofeno, un fármaco con múltiples enlaces rotables.

**Tarea:**
1. SMILES del ibuprofeno: `CC(C)Cc1ccc(cc1)C(C)C(=O)O`
2. Genera 200 conformaciones
3. Determina cuántos enlaces rotables tiene
4. ¿Cuántas conformaciones están dentro de 3 kcal/mol del mínimo?
5. Calcula la diversidad conformacional (rango de energías)
6. ¿Qué porcentaje de la población total representan las 10 conformaciones más estables?

In [ ]:
# TU CÓDIGO AQUÍ
# Análisis conformacional del ibuprofeno

### Ejercicio 4: Comparación de Campos de Fuerza

Compara los resultados obtenidos con diferentes campos de fuerza.

**Tarea:**
1. Genera 50 conformaciones de n-butano usando MMFF94
2. Optimiza las mismas conformaciones con UFF
3. Compara las energías obtenidas
4. ¿Cuál es la correlación entre ambos métodos?
5. ¿Cambia la conformación de menor energía?

*Pista:* Usa `AllChem.UFFOptimizeMolecule()` para UFF

In [ ]:
# TU CÓDIGO AQUÍ
# Compara MMFF94 vs UFF

### Ejercicio 5: Efecto de la Temperatura

Investiga cómo cambia la distribución de poblaciones con la temperatura.

**Tarea:**
1. Usa las conformaciones de ciclohexano del Ejercicio 1
2. Calcula poblaciones de Boltzmann a:
   - 200 K (baja temperatura)
   - 298 K (temperatura ambiente)
   - 500 K (alta temperatura)
3. Grafica la población del confórmero más estable vs temperatura
4. ¿Cómo afecta la temperatura a la diversidad conformacional?
5. ¿A qué temperatura el segundo confórmero más estable representa el 10% de la población?

In [ ]:
# TU CÓDIGO AQUÍ
# Analiza el efecto de la temperatura en las poblaciones

## Resumen y Conclusiones

En esta actividad has aprendido a:

✅ **Generar conformaciones** usando el algoritmo ETKDG de RDKit  
✅ **Optimizar geometrías** con campos de fuerza MMFF94 y UFF  
✅ **Calcular poblaciones** usando la distribución de Boltzmann  
✅ **Analizar ángulos diedros** para caracterizar conformaciones  
✅ **Identificar conformaciones de baja energía** relevantes biológicamente  
✅ **Visualizar y comparar** múltiples conformaciones  

### Puntos Clave

- Las conformaciones afectan **propiedades moleculares** y **actividad biológica**
- La búsqueda conformacional es **esencial** en docking molecular y diseño de fármacos
- Las poblaciones de Boltzmann predicen la **distribución en equilibrio**
- Conformaciones dentro de **2-3 kcal/mol** del mínimo son relevantes a temperatura ambiente
- Métodos estocásticos (ETKDG) son más eficientes para moléculas complejas

### Próximos Pasos

- **Actividad 2.6:** Descriptores moleculares y propiedades calculadas
- **Actividad 2.7:** Búsqueda y descarga de estructuras desde bases de datos

---

## Referencias

1. Riniker, S., & Landrum, G. A. (2015). *Better Informed Distance Geometry: Using What We Know To Improve Conformation Generation.* J. Chem. Inf. Model., 55(12), 2562-2574.

2. Hawkins, P. C. (2017). *Conformation Generation: The State of the Art.* J. Chem. Inf. Model., 57(8), 1747-1756.

3. RDKit Documentation: [Conformer Generation](https://www.rdkit.org/docs/GettingStartedInPython.html#working-with-3d-molecules)

4. Leach, A. R. (2001). *Molecular Modelling: Principles and Applications* (2nd ed.). Pearson Education.

5. Halgren, T. A. (1996). *Merck molecular force field.* J. Comput. Chem., 17(5-6), 490-519.

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 2.5: Generación de Conformaciones Moleculares**

**Siguiente actividad**: Descriptores Moleculares y Propiedades Calculadas

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_2.4-Visualización_3D-blue.svg)](04_visualizacion_3d.ipynb)
[![Siguiente](https://img.shields.io/badge/Actividad_2.6_➡️-Descriptores-green.svg)](06_descriptores.ipynb)

---

📚 **[Volver al Módulo 2](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>